In [1]:
!pip install gurobipy

You should consider upgrading via the 'C:\Users\artur\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


## A Facility Location Problem

![slide1.png](https://i.ibb.co/q1G74jk/download-2.png)

![slide3.png](https://i.ibb.co/v4H51g7/download-3.png)

The prob variable is created using the LpProblem function, with the usual input parameters.

In [2]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [3]:
p = 1 #P is the number of facilities that we want to locate.

We create a list of our facilities.

In [4]:
Facilities = ['A', 'B']

To define our customer id's we use a python generator to generate a number for each node

In [5]:
Customers = [str(i+1) for i in range(4)]
Customers

['1', '2', '3', '4']

In [6]:
#product creates the cartesian product from 2 or more lists.
customer_facilities = list(product(Customers, Facilities))
customer_facilities

[('1', 'A'),
 ('1', 'B'),
 ('2', 'A'),
 ('2', 'B'),
 ('3', 'A'),
 ('3', 'B'),
 ('4', 'A'),
 ('4', 'B')]

Now we provide the customer demand in the order of our customer ids.

In [7]:
demand = [10, 11, 2, 3]

Just to make sure we generated a label for each node and put in a supply for each node we include an assert.

In [8]:
assert len(demand) == len(Customers)

We can now convert this into a dictionary of values, with customer IDs used as keys.

In [9]:
D = dict(zip(Customers, demand))
D

{'1': 10, '2': 11, '3': 2, '4': 3}

We give the distance between customers and facilities as a dictionary with the tuple (customer, facility) as kyy. The cost from customer i to facility location j can be retrieved with C\[(i,j)]

In [10]:
C = {('1', 'A'): 20,
 ('1', 'B'): 30,
 ('2', 'A'): 10,
 ('2', 'B'): 20,
 ('3', 'A'): 10,
 ('3', 'B'): 10,
 ('4', 'A'): 10,
 ('4', 'B'): 20}

In [11]:
#Define model
m = gp.Model('facility_location')

Restricted license - for non-production use only - expires 2024-10-28


We add decision variables if we locate a facility at candidate site j or not. Setting the variable type as BINARY the lower bound is 0 and upper bound is 1, for yes or no.

In [12]:
X_j = m.addVars(Facilities, vtype=GRB.BINARY, name='x')
X_j

{'A': <gurobi.Var *Awaiting Model Update*>,
 'B': <gurobi.Var *Awaiting Model Update*>}

We add decision variables if we service customer i from candidate site j or not. Setting the variable type as BINARY the lower bound is 0 and upper bound is 1, for yes or no.

In [13]:
Y_ij = m.addVars(customer_facilities, vtype=GRB.BINARY, name='y')
Y_ij

{('1', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('1', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('2', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('2', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('3', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('3', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('4', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('4', 'B'): <gurobi.Var *Awaiting Model Update*>}

:First we add the objective function to the problem. The sum of actual distance for the customer demand is the distance between facilities and customers and whether we plan the facility times the demand for the customer.

In [14]:
# The second or (third) argument filters to a sub product of only decision varibles
# were the first value in the key is equal to c
# Y_ij.prod(C,'1') is the product of Y_ij and C where i is '1'
# Y_ij.prod(C,'1','A') is the product of Y_ij and c where i is '1' and j is 'A'
# Y_ij.prod(C,'*','A') is the product of Y_ij and c where i is any value and j is 'A'
# We then sum up the product of each customer using quicksum
m.setObjective(gp.quicksum([Y_ij.prod(C,c) * D[c] for c in Customers]), GRB.MINIMIZE)

Our first constraint is that each customer is served by one and only one facility.

In [15]:
m.addConstrs((Y_ij.sum(customer) == 1 for customer in Customers),
              name='1FacilityPerCustomer')

{'1': <gurobi.Constr *Awaiting Model Update*>,
 '2': <gurobi.Constr *Awaiting Model Update*>,
 '3': <gurobi.Constr *Awaiting Model Update*>,
 '4': <gurobi.Constr *Awaiting Model Update*>}

Our second constraint is that we want to restrict the number of factilities to p

In [16]:
m.addConstr(X_j.sum() == p, name='LimitFacilitiesToP')

<gurobi.Constr *Awaiting Model Update*>

Our third constraint is that a customer can only be served from facility j, if we planned a facility there.

In [17]:
for customer, facility in customer_facilities:
    m.addConstr(Y_ij[(customer, facility)] - X_j[facility] <= 0,name=f"OnlyAllowServiceAt{customer}IfFacility{facility}")

Recall:
 - if we service customer i from candidate site j: Y_ij = 1 , otherwise Y_ij = 0
 - if we locate a facility at candidate site j: X_j = 1, otherwise X_j = 0

So the following situations are all allowed:
 - when we locate a facility at candidate site X_j, then if we service customer i from there we have: 1 - 1 = 0
 - when we locate a facility at candidate site X_j, then if we do NOT service customer i from there we have: 0 - 1 = -1
 - when we do NOT locate a facility at candidate site X_j, then we can not service any customers from there and we have: 0 - 0 = 0

 And the following situation is not allowed:
 - when we do NOT locate a facility at candidate site X_j, servicing a customer from there would give: 1 - 0 = 1 (which is contrained not to be allowed)

Finally, recall:

- The constraints that x_ij and x_j have to be {0,1} is specified by the lowBound and upBound parameteer when we introduced the decision variables.

In [21]:
m.write('FacilityLocationProblem.lp')

Finally we can solve our problem.

In [22]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 10 columns and 26 nonzeros
Model fingerprint: 0x93ef3f78
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]


Presolve removed 13 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 360 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+02, best bound 3.600000000000e+02, gap 0.0000%


In [23]:
print(f"Optimal objective value: {m.objVal}")

for plant in X_j.keys():
    if (abs(X_j[plant].x) > 0): #Only print if not 0
        print(f"Build a facility at location {plant}.")
print("\nShipment plan:")
for customer, facility in Y_ij.keys():
    if (abs(Y_ij[customer, facility].x) > 0): #Only print if not 0
        print (f"Service customer {customer} from facility {facility}")

Optimal objective value: 360.0
Build a facility at location A.

Shipment plan:
Service customer 1 from facility A
Service customer 2 from facility A
Service customer 3 from facility A
Service customer 4 from facility A


Our solution is to place the facility at location A for a mimimal transportation distance of 360